In [27]:
import numpy as np

In [28]:
class Node:
  def __init__(self,feature=None, threshold=None, left=None, right=None,value=None):
    self.feature = feature
    self.left = left
    self.threshold = threshold
    self.right = right
    self.value = value

    '''
    Notes regarding the class:-
    1. value attribute is only for final leaf node that gives out a value
    2. left: for samples satisfying
    3. right: for samples not satisfying
    4. feature and threshiold together is a condition (age<30)
    '''

In [48]:
class Classifier:
  def __init__(self,max_depth=None,samplesPerLeaf=1):
    self.max_depth=max_depth
    self.samplesPerLeaf=samplesPerLeaf
    self.root=None

  def gini_impurity(self,y):
    if(len(y)==0):
      return 0
    # if there no points located within that region, then gini=0

    class_counts={}
    for label in y:
      if label not in class_counts:
        class_counts[label]=0
      class_counts[label]+=1

    sum=0;
    for count in class_counts.values():
      prob = count/len(y)
      sum+=prob**2
    return 1-sum

  # gini = 1-(sum of squares of each prob)
  def fit(self, X, y):
    self.num_classes = len(np.unique(y))
    self.num_dims = X.shape[1]
    self.root = self.make_tree(X, y, 0)

  def best_split(self, X, y):
    m,n = X.shape
    # m is the number of records
    # n is the number of dims

    if m<=1:
      return None, None

    # ie. You cant actually split if there is only 1/0 nodes

    min_gini = 1
    best_feature = None
    best_threshold = None

    for feature in range(self.num_dims):
      thresholds = np.unique(X[:, feature])

      for threshold in thresholds:

        left_idx = []

        for i in range(m):
          if(X[i][feature]<=threshold):
            left_idx.append(i)

        if len(left_idx) < self.samplesPerLeaf or (m - len(left_idx)) < self.samplesPerLeaf:
          continue

        left_y = []
        right_y = []
        for i in range(m):
            if i in left_idx:
                left_y.append(y[i])
            else:
                right_y.append(y[i])

        num_left = len(left_y)
        num_right = len(right_y)
        left_gini = self.gini_impurity(left_y)
        right_gini = self.gini_impurity(right_y)

        weighted_gini = (num_left / m) * left_gini + (num_right / m) * right_gini

        if weighted_gini < min_gini:
          min_gini = weighted_gini
          best_feature = feature
          best_threshold = threshold

    return best_feature, best_threshold

  def make_tree(self, X, y, depth=0):
    num_samples, num_features = X.shape
    num_classes = len(np.unique(y))

    if(self.max_depth is not None and depth>=self.max_depth):
      leaf_value = self.most_common_label(y)
      return Node(value=leaf_value)

    if(self.samplesPerLeaf is not None and num_samples<=self.samplesPerLeaf):
      leaf_value = self.most_common_label(y)
      return Node(value=leaf_value)

    if num_classes==1:
      leaf_value = self.most_common_label(y)
      return Node(value = leaf_value)

    # Base cases (3) have been written
    # Recursive function follows

    best_feature, best_threshold = self.best_split(X, y)

    if best_feature is None:
      leaf_value = self.most_common_label(y)
      return Node(value=leaf_value)

    # Another base case

    left_idx = []

    for i in range(num_samples):
      if(X[i][best_feature]<=best_threshold):
        left_idx.append(i)

    if len(left_idx) < self.samplesPerLeaf or (num_samples - len(left_idx)) < self.samplesPerLeaf:
        leaf_value = self.most_common_label(y)
        return Node(value=leaf_value)


    left_X = []
    left_y = []
    right_X = []
    right_y = []

    for i in range(num_samples):
        if i in left_idx:
            left_X.append(X[i])
            left_y.append(y[i])
        else:
            right_X.append(X[i])
            right_y.append(y[i])

    # Convert back to numpy arrays
    left_X = np.array(left_X)
    left_y = np.array(left_y)
    right_X = np.array(right_X)
    right_y = np.array(right_y)

    # Recurse
    left_child = self.make_tree(left_X, left_y, depth+1)
    right_child = self.make_tree(right_X, right_y, depth+1)

    return Node(best_feature, best_threshold, left_child, right_child)

  def most_common_label(self, y):
    num1=0;
    num0=0;

    for i in y:
      if(i==1):
        num1+=1
      else:
        num0+=1

    if(num1>num0):
      return 1
    else:
      return 0

  def predict(self, x):
    node = self.root
    while node.value is None:
      if(x[node.feature]<=node.threshold):
        node = node.left
      else:
        node = node.right
    return node.value

  def print_tree(self, node=None, indent=""):
        if node is None:
            node = self.root

        # If leaf node, print value
        if node.value is not None:
            print(f"{indent}Predict: {'Yes' if node.value == 1 else 'No'}")
            return

        # Print decision criteria
        print(f"{indent}Feature {node.feature} <= {node.threshold}")

        # Print left subtree
        print(f"{indent}Left:")
        self.print_tree(node.left, indent + "  ")

        # Print right subtree
        print(f"{indent}Right:")
        self.print_tree(node.right, indent + "  ")
  def predict_multiple_samples(self, X):
    predictions = []
    for x in X:
        predictions.append(self.predict(x))
    return np.array(predictions)

In [49]:

data = {
    'Age': [25, 30, 35, 40, 45, 50, 55, 60],
    'Income': ['High', 'High', 'Medium', 'Low', 'Low', 'Low', 'Medium', 'High'],
    'Student': ['No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No'],
    'Credit_Rating': ['Fair', 'Excellent', 'Fair', 'Fair', 'Fair', 'Excellent', 'Excellent', 'Fair'],
    'Buy_Computer': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No']
}



In [50]:
def encode_features(data):
    X = np.zeros((len(data['Age']), 4))
    student_map = {'No': 0, 'Yes': 1}
    credit_map = {'Fair': 0, 'Excellent': 1}
    income_map = {'Low': 0, 'Medium': 1, 'High': 2}

    X[:, 0] = data['Age']
    X[:, 1] = [income_map[x] for x in data['Income']]
    X[:, 2] = [student_map[x] for x in data['Student']]
    X[:, 3] = [credit_map[x] for x in data['Credit_Rating']]

    y = np.array([1 if val == 'Yes' else 0 for val in data['Buy_Computer']])

    return X, y

# Encode the features
X, y = encode_features(data)


In [51]:
fennekin = Classifier(max_depth=3,samplesPerLeaf=1)
fennekin.fit(X,y)
fennekin.print_tree()

Feature 1 <= 1.0
Left:
  Feature 0 <= 45.0
  Left:
    Predict: Yes
  Right:
    Feature 0 <= 50.0
    Left:
      Predict: No
    Right:
      Predict: Yes
Right:
  Predict: No


In [52]:
# Accuracy measuremnet on training error

predictions = fennekin.predict_multiple_samples(X)
accuracy = np.mean(predictions == y)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 100.00%


In [47]:
# Prediction a test sample

test_X = [42, 0, 0, 1]
test_Y = fennekin.predict(test_X)
print(test_Y)


1


In [53]:
# Bagging with OOB error
import numpy as np
from random import randrange

class BaggedClassifier:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_leaf=1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.trees = []
        self.oob_score = None

    def fit(self, X, y):
        n_samples = len(X)
        self.oob_predictions = {i: [] for i in range(n_samples)}

        for _ in range(self.n_estimators):
            b_indices = []
            for _ in range(n_samples):
                b_indices.append(randrange(n_samples))


            b_set = set(b_indices)
            oob_indices = [i for i in range(n_samples) if i not in b_set]


            X_b = []
            y_b = []
            for idx in b_indices:
                X_b.append(X[idx])
                y_b.append(y[idx])


            X_b = np.array(X_b)
            y_b = np.array(y_b)


            tree = Classifier(max_depth=self.max_depth, samplesPerLeaf=self.min_samples_leaf)
            tree.fit(X_b, y_b)
            self.trees.append(tree)


            for idx in oob_indices:
                prediction = tree.predict(X[idx])
                self.oob_predictions[idx].append(prediction)

        oob_errors = 0
        oob_total = 0

        for idx, predictions in self.oob_predictions.items():
            if len(predictions) > 0:
                counts = {}
                for pred in predictions:
                    if pred not in counts:
                        counts[pred] = 0
                    counts[pred] += 1

                majority_vote = max(counts, key=counts.get)
                if majority_vote != y[idx]:
                    oob_errors += 1
                oob_total += 1


        if oob_total > 0:
            self.oob_score = 1 - (oob_errors / oob_total)

        return self

    def predict(self, X):
        predictions = []

        for sample in X:

            votes = []
            for tree in self.trees:
                votes.append(tree.predict(sample))


            counts = {}
            for vote in votes:
                if vote not in counts:
                    counts[vote] = 0
                counts[vote] += 1


            majority_class = max(counts, key=counts.get)
            predictions.append(majority_class)

        return np.array(predictions)

In [54]:
bag = BaggedClassifier(10,3,1)
bag.fit(X,y)
# Printing scores
print(f"OOB Score: {bag.oob_score * 100:.2f}%")
print(f"OOb Error: {1-bag.oob_score: .2f}")

OOB Score: 37.50%
OOb Error:  0.62


In [55]:
import numpy as np
from random import randrange, sample

class RandomForestClassifier:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_leaf=1, max_features=2):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features  # Number of features to consider for each split
        self.trees = []
        self.oob_score = None

    def fit(self, X, y):
        n_samples = len(X)
        n_features = X.shape[1]

        self.max_features = min(self.max_features, n_features)

        self.oob_predictions = {i: [] for i in range(n_samples)}

        for _ in range(self.n_estimators):
            b_indices = []
            for _ in range(n_samples):
                b_indices.append(randrange(n_samples))

            b_set = set(b_indices)
            oob_indices = [i for i in range(n_samples) if i not in b_set]

            X_b = []
            y_b = []
            for idx in b_indices:
                X_b.append(X[idx])
                y_b.append(y[idx])

            X_b = np.array(X_b)
            y_b = np.array(y_b)

            tree = RandomTree(max_depth=self.max_depth,
                             samplesPerLeaf=self.min_samples_leaf,
                             max_features=self.max_features)
            tree.fit(X_b, y_b)
            self.trees.append(tree)

            for idx in oob_indices:
                prediction = tree.predict(X[idx])
                self.oob_predictions[idx].append(prediction)

        oob_errors = 0
        oob_total = 0

        for idx, predictions in self.oob_predictions.items():
            if len(predictions) > 0:
                counts = {}
                for pred in predictions:
                    if pred not in counts:
                        counts[pred] = 0
                    counts[pred] += 1

                majority_vote = max(counts, key=counts.get)
                if majority_vote != y[idx]:
                    oob_errors += 1
                oob_total += 1

        if oob_total > 0:
            self.oob_score = 1 - (oob_errors / oob_total)

        return self

    def predict(self, X):
        predictions = []

        for sample in X:
            votes = []
            for tree in self.trees:
                votes.append(tree.predict(sample))

            counts = {}
            for vote in votes:
                if vote not in counts:
                    counts[vote] = 0
                counts[vote] += 1

            majority_class = max(counts, key=counts.get)
            predictions.append(majority_class)

        return np.array(predictions)

class RandomTree(Classifier):
    def __init__(self, max_depth=None, samplesPerLeaf=1, max_features=2):
        super().__init__(max_depth, samplesPerLeaf)
        self.max_features = max_features

    def best_split(self, X, y):
        m, n = X.shape
        if m <= 1:
            return None, None

        min_gini = 1
        best_feature = None
        best_threshold = None

        feature_indices = list(range(self.num_dims))
        if len(feature_indices) > self.max_features:
            feature_indices = sample(feature_indices, self.max_features)

        for feature in feature_indices:
            thresholds = np.unique(X[:, feature])

            for threshold in thresholds:
                left_idx = []
                for i in range(m):
                    if X[i][feature] <= threshold:
                        left_idx.append(i)

                if len(left_idx) < self.samplesPerLeaf or (m - len(left_idx)) < self.samplesPerLeaf:
                    continue

                left_y = []
                right_y = []
                for i in range(m):
                    if i in left_idx:
                        left_y.append(y[i])
                    else:
                        right_y.append(y[i])

                num_left = len(left_y)
                num_right = len(right_y)
                left_gini = self.gini_impurity(left_y)
                right_gini = self.gini_impurity(right_y)

                weighted_gini = (num_left / m) * left_gini + (num_right / m) * right_gini

                if weighted_gini < min_gini:
                    min_gini = weighted_gini
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

In [56]:
fennekin2 = RandomForestClassifier(10,3,1,2)
fennekin2.fit(X,y)
# Printing scores
print(f"OOB Score: {fennekin2.oob_score * 100:.2f}%")
print(f"OOb Error: {1-fennekin2.oob_score: .2f}")

OOB Score: 42.86%
OOb Error:  0.57
